# Importing Libraries


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.16.1'

# Part-1 Data PreProcessing

PreProcessing the Training Set

In [3]:
# To avoid Overfitting or Overlearning of CNN on these images, we'll apply tranformations on images='Image Augmentation'
train_datagen = ImageDataGenerator(rescale = 1./255, # represents feature-scaling
                                   shear_range = 0.2, 
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/Users/aaku/Desktop/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary') # Mode of outcome(Cat/Dog)

Found 8000 images belonging to 2 classes.


PreProcessing the Test Set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255) # only feature scaling on test_set because 
# they are new to test the model and like we earlier used fit_transform in training_set but only transform in test_set, 
# so that there are no leakages in information of test_images.
test_set = test_datagen.flow_from_directory('/Users/aaku/Desktop/dataset/test_set',
                                           target_size=(64, 64),
                                           batch_size = 32,
                                           class_mode = 'binary')

Found 2000 images belonging to 2 classes.


# Part-2 Building CNN

Initializing the CNN

In [5]:
cnn = tf.keras.models.Sequential() 

Step-1 Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3])) 
# filters=32 --> As in most common architectures and input_shape=[64,64-->(We entered target_size of (64,64)), 3-->RGB Img.]

/Users/aaku/anaconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Step-2 Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# strides refer to the step size or the shift applied to the pooling window as it traverses the input feature map.

Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step-3 Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

Step-4 Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step-5 Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # Binary Classification

# Part-3 Training the CNN

Compiling the CNN

In [12]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the CNN over training_set and evaluating it on test_set 

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


/Users/aaku/anaconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.5552 - loss: 0.6902 - val_accuracy: 0.5035 - val_loss: 0.8394
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.6299 - loss: 0.6503 - val_accuracy: 0.6855 - val_loss: 0.5871
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - accuracy: 0.6785 - loss: 0.5925 - val_accuracy: 0.7130 - val_loss: 0.5633
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7169 - loss: 0.5513 - val_accuracy: 0.7390 - val_loss: 0.5318
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7413 - loss: 0.5208 - val_accuracy: 0.7125 - val_loss: 0.5690
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.7498 - loss: 0.5062 - val_accuracy: 0.7400 - val_loss: 0.5184
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7730 - loss: 0.4739 - val_accuracy: 0.7630 - val_loss: 0.5068
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - accuracy: 0.7734 - loss: 0.4664 - val_accurac

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/Users/aaku/Desktop/dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
# If test_image is a single image represented as a 2D array (height, width) or a 3D array (height, width, channels), 
# it does not have the required batch dimension. To make it compatible with the expected input shape, 
# we need to add an extra dimension to represent the batch size.

result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1: # result[batch][element of the batch]
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

In [ ]:
# If Image-Augmentation wouldn't have been there, then training-accuracy would have been very high aroung 97% and 
# test-accuracy would have been lower around 50%...Therefore, Image-Augmentation is necessary.